# Лекция 3.2. Pandas

Pandas — это библиотека Python, предоставляющая широкие возможности для анализа данных. Данные, с которыми работают датасаентисты, часто хранятся в форме табличек — например, в форматах .csv, .tsv или .xlsx. С помощью библиотеки Pandas такие табличные данные очень удобно загружать, обрабатывать и анализировать с помощью SQL-подобных запросов. А в связке с библиотеками Matplotlib и Seaborn Pandas предоставляет широкие возможности визуального анализа табличных данных.


Основными структурами данных в Pandas являются классы Series и DataFrame. Первый из них представляет собой одномерный индексированный массив данных некоторого фиксированного типа. Второй – это двухмерная структура данных, представляющая собой таблицу, каждый столбец которой содержит данные одного типа. Можно представлять её как словарь объектов типа Series. Структура DataFrame отлично подходит для представления реальных данных: строки соответствуют признаковым описаниям отдельных объектов, а столбцы соответствуют признакам.

## Импорт модуля pandas и numpy

In [0]:
import pandas as pd
import numpy as np

## Обзор основных методов

Будем показывать основные методы в деле, анализируя набор данных по оттоку клиентов телеком-оператора, который уже закачан в [репозиторий](data/telecom_churn.csv) и доступен по относительному пути data/telecom_churn.csv. Прочитаем данные (метод read_csv) и посмотрим на первые 5 строк с помощью метода head:

In [1]:
df = pd.read_csv('data/telecom_churn.csv')

NameError: ignored